# Plotting Ballistic Ejecta from the South Pole-Aitken Impact

## Workflow:
This script runs through all the tracers in the data file and selects the tracers that follow a ballistic trajectory. It outputs the results to a csv file which can be used for plotting.

In [1]:
#import python packages 
import tools
import numpy as np
import pandas as pd

In [3]:
#import the data file. Change the path to your file :) 
data = tools.load_ballistic_data("/Users/namya/Desktop/tracer_export_all.npz", end = 400)
# print(data['ymark'].max())

In [4]:
#get data
velocity_data = tools.get_velocity(data)
radial_distance_data = tools.get_radial_distance(data)
radial_velocity_data = tools.get_radial_velocity(data)
radial_acceleration_data = tools.get_radial_acceleration(data, radial_velocity_data)
average_time = tools.get_average_time(data)

In [5]:
h_launch = 150
tracer_idxs, time_steps = tools.BallisticCheckVec(data, h_launch)
# print(np.min(time_steps))
# print(np.max(time_steps))
# print(time_steps.shape)
ejection_times = data['time'][time_steps]

In [6]:
tracer_material = data['trm'][tracer_idxs]

In [7]:
R0 = 1750
g0 = 0.00162

g = g0 * R0**2 / (R0 + h_launch) **2
v_tilda = tools.get_v_tilda_vec(velocity_data, tracer_idxs, time_steps, R0 + h_launch, g)

In [8]:
v_r, longitude, latitude = tools.get_launch_vector_vec(data,velocity_data, tracer_idxs, time_steps)
longitude_magnitude, longitude_direction = longitude
latitude_magnitude, latitude_direction = latitude

In [9]:
theta_launch, height, distance, azimuth = tools.get_ballistic_vectors_vec(data, velocity_data, tracer_idxs, time_steps, h_launch, R0, g0)

/Users/namya/year4_project/tools/ballistics.py:105: RuntimeWarning: invalid value encountered in arccos
  beta = np.arccos(data['zmark'] / (data['r']+ 1e-12))[:, :-1]  # angle at the centre of the moon


In [43]:
#multiply the azimuth with -1 to get the right set up
final_latitude, final_longitude = tools.track_vec(latitude[0], longitude[0], -1 * azimuth, distance)

In [44]:
def make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth):
    initial_df = pd.DataFrame({'Tracer Index' : tracer_idxs, 'Ejection Time' : ejection_times, 'Resultant Velocity' : v_r, 'Launch Longitude' : longitude[0], 'Launch Latitude' : latitude[0], 'Launch Angle' : theta_launch, 'Height' : height, 'Distance' : distance, 'Landing Longitude' : final_longitude, 'Landing Latitude' : final_latitude, 'Tracer Material' : tracer_material, 'Azimuth' : azimuth})
    return initial_df

In [45]:
df = make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth)
df
# print(df['Launch Longitude'][df['Launch Longitude'] < 150])
# print(df['Ejection Time'][df['Ejection Time'] > 0].min())
# print(df.Distance)
# print(df.Distance.describe())
# print(df[df.Distance < 0])
# # print(df['Ejection Time'].describe())
# print(df.loc[1360])
# print(df['Resultant Velocity'].describe())
# # print(df['Tracer Material'].value_counts())
# print(df['Resultant Velocity'].describe())

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth
0,0,112.025597,3.567570,178.928060,68.623144,53.780778,-4165.526625,3747.565530,176.398754,-54.059631,4.0,178.236196
1,1,110.039398,3.675897,178.956358,68.769574,52.755341,-4083.874445,3690.357295,176.576237,-52.041548,4.0,178.295441
2,2,210.030502,1.564486,178.973916,63.860490,43.234086,1115.458044,2748.912389,178.320432,-26.138652,4.0,179.413350
3,4,146.047302,3.035805,178.783652,66.573687,54.054510,-5243.161668,3877.581422,178.783652,-60.380056,4.0,180.000000
4,5,136.046997,2.775873,178.908277,67.246903,52.478551,-6752.403061,3906.837386,176.511662,-60.652654,4.0,178.511486
...,...,...,...,...,...,...,...,...,...,...,...,...
67953,524251,404.081696,0.828934,170.547593,58.113793,40.613468,132.819803,750.660985,166.103106,33.719464,2.0,171.084701
67954,524252,700.052612,0.408345,161.875505,54.655800,7.879490,1.217090,1118.995015,144.350702,20.507410,2.0,151.794733
67955,524280,296.067291,1.329092,175.046999,60.703975,52.199425,705.097133,1644.611834,170.230194,6.980468,2.0,174.075017
67956,524281,374.084106,0.938261,169.255493,59.026070,44.094048,205.951240,914.207949,161.238375,29.600587,2.0,165.934169


In [46]:
#exclude tracers where their velocity exceeds the escape velocity of the moon
df = df[df['Resultant Velocity']< 2.38] 

In [49]:
#save the dataframe as a csv file 
df.to_csv("full_tracer.csv", index=False)    